In [1]:
import os
from pathlib import Path

path = Path('model.pkl')

!pip install -Uqq fastai

In [2]:
print(f"File exists: {path.exists()}")
print(f"File size: {path.stat().st_size if path.exists() else 'N/A'} bytes")
print(f"Full path: {path.absolute()}")
import zipfile

try:
    with zipfile.ZipFile(path, 'r') as zip_file:
        print("File is a valid ZIP archive")
except zipfile.BadZipFile:
    print("File is corrupted or not a valid ZIP archive")
except FileNotFoundError:
    print("File not found")


File exists: True
File size: 102992994 bytes
Full path: /Users/kalanjarvis-loewen/Desktop/Coding/learning/ml/LeanAI/core/model.pkl
File is a valid ZIP archive


In [4]:
import torch
from PIL import Image
import torchvision.transforms as transforms
from fastai.vision.all import *

# Load model and extract the PyTorch part
model = load_learner('model.pkl')
pytorch_model = model.model
pytorch_model.eval()

# Manual preprocessing for 192x192 images
transform = transforms.Compose([
    transforms.Resize((192, 192)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Process image and predict
img = Image.open('images/0_image_1.jpg').convert('RGB')
img_tensor = transform(img).unsqueeze(0)

with torch.no_grad():
    output = pytorch_model(img_tensor)
    prediction = output[0][0].item()
    print(f"Bodyfat prediction: {prediction:.4f}")

Bodyfat prediction: 8.1319


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [11]:
def preprocess_img(pre_img):
    img = Image.open('images/0_image_1.jpg').convert('RGB')
    return transform(img).unsqueeze(0)

In [12]:
def predict_image(img):

    if img is None:
        return "Please upload an image first"
    
    try:
        print(f"Input type: {type(img)}")

        prediction = None
        
        with torch.no_grad():
            output = pytorch_model(preprocess_img(img))
            prediction = output[0][0].item()
            print(f"Bodyfat prediction: {prediction:.4f}")
        
        
        return float(prediction)
        
    except Exception as e:
        return print(f"error: {e}")

In [13]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Body Fat Prediction")
    
    with gr.Row():
        image_input = gr.Image(label="Upload Image")
        output = gr.Number(label="Predicted Value")
    
    predict_btn = gr.Button("Predict")
    predict_btn.click(predict_image, inputs=image_input, outputs=output)

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://3056108411420f1034.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Input type: <class 'numpy.ndarray'>
Bodyfat prediction: 8.1319
Input type: <class 'numpy.ndarray'>
Bodyfat prediction: 8.1319
Input type: <class 'numpy.ndarray'>
Bodyfat prediction: 8.1319
Input type: <class 'numpy.ndarray'>
Bodyfat prediction: 8.1319
Input type: <class 'numpy.ndarray'>
Bodyfat prediction: 8.1319
Input type: <class 'numpy.ndarray'>
Bodyfat prediction: 8.1319
Input type: <class 'numpy.ndarray'>
Bodyfat prediction: 8.1319
